In [ ]:
import copy
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from numpy import hstack
from sklearn.preprocessing import StandardScaler
import datetime
import time
import joblib
from datetime import timedelta, date
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from numpy import array
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import os
import seaborn as sns; sns.set_theme()
import errno
from keras.layers import Flatten
from keras.layers import ConvLSTM2D
from keras.models import load_model
import pickle
from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report, confusion_matrix


from collections import Counter

import collections
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from matplotlib import pyplot

In [ ]:
pip install cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 44.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd

def load_data_south_indian(url):
    """
    Load and preprocess data for South Indian region.

    Parameters:
    - url (str): URL or file path to the CSV file.

    Returns:
    - pd.DataFrame: Processed DataFrame with a 'category' column.
    """
    # Load data from CSV
    df = pd.read_csv(url)

    # Add a 'category' column based on speed ranges
    df['category'] = df['Speed(knots)'].apply(lambda x:
        0 if x <= 33 else
        1 if 34 <= x <= 47 else
        2 if 48 <= x <= 63 else
        3 if 64 <= x <= 89 else
        4 if 90 <= x <= 115 else
        5
        )

    return df

def load_data_south_pacific(url):
    """
    Load and preprocess data for South Pacific region.

    Parameters:
    - url (str): URL or file path to the CSV file.

    Returns:
    - pd.DataFrame: Processed DataFrame with a 'category' column.
    """
    # Load data from CSV
    df = pd.read_csv(url)

    # Add a 'category' column based on speed ranges
    df['category'] = df['Speed(knots)'].apply(lambda x:
        0 if x <= 33 else
        1 if 34 <= x <= 47 else
        2 if 48 <= x <= 63 else
        3 if 64 <= x <= 85 else
        4 if 86 <= x <= 107 else
        5
    )
    return df

In [ ]:
ocean = 'south_indian'

# Display the selected ocean
print(f"Selected ocean: {ocean}")

# Set the data URL and function based on the selected ocean
if ocean == 'south_indian':
    url_data = 'https://raw.githubusercontent.com/sydney-machine-learning/cyclonedatasets/main/SouthIndian-SouthPacific-Ocean/South_indian_hurricane.csv'
    data_loading_function = load_data_south_indian
    hot_encoded_result_file_name = 'south_indian'
    category_result_file_name = 'roc_data_south_indian'
else:
    url_data = 'https://raw.githubusercontent.com/sydney-machine-learning/cyclonedatasets/main/SouthIndian-SouthPacific-Ocean/South_pacific_hurricane.csv'
    data_loading_function = load_data_south_pacific
    hot_encoded_result_file_name = 'south_pacific'
    category_result_file_name = 'roc_data_south_pacific'

# Display the data URL for verification
print(f"Data URL: {url_data}")

# Display the result file names
print(f"Hot-encoded result file name: {hot_encoded_result_file_name}")
print(f"Category result file name: {category_result_file_name}")


Selected ocean: south_indian
Data URL: https://raw.githubusercontent.com/sydney-machine-learning/cyclonedatasets/main/SouthIndian-SouthPacific-Ocean/South_indian_hurricane.csv
Hot-encoded result file name: south_indian
Category result file name: roc_data_south_indian


In [ ]:
# Load data using the specified function and URL
df = data_loading_function(url_data)

# Latitude and longitude values into int
df['Lat'] = df['Lat'].apply(lambda x: -int(x[:-1]) * 0.1 if x.endswith('N') else int(x[:-1]) * 0.1 if x.endswith('S') else 0)
df['Lon'] = df['Lon'].apply(lambda x: -int(x[:-1]) * 0.1 if x.endswith('W') else int(x[:-1]) * 0.1 if x.endswith('E') else 0)

# Extract 'Speed(knots)' and 'category' columns as lists
speed = df['Speed(knots)'].tolist()
categories = df['category'].tolist()
latitude = df['Lat'].tolist()
longitude = df['Lon'].tolist()


# Display the first few rows of the DataFrame
df_head = df.head()
print("DataFrame Head:")
print(df_head)

DataFrame Head:
  Basin  No. of Cycl        Time     V5  V6   Lat   Lon  Speed(knots)  \
0    SI            1  1981072512   BEST   0  11.8  86.7            20   
1    SI            1  1981072518   BEST   0  11.6  86.4            25   
2    SI            1  1981072600   BEST   0  11.3  86.2            25   
3    SI            1  1981072606   BEST   0  10.9  86.2            25   
4    SI            1  1981072612   BEST   0  10.4  86.1            25   

   lat_tenth  lon_tenth  category  
0       11.8       86.7         0  
1       11.6       86.4         0  
2       11.3       86.2         0  
3       10.9       86.2         0  
4       10.4       86.1         0  


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def split_sequence(sequences, n_steps_in, n_steps_out):
    """
    Split multivariate sequences into input and output parts.

    Parameters:
    - sequences (numpy.ndarray): Multivariate time series data.
    - n_steps_in (int): Number of input time steps.
    - n_steps_out (int): Number of output time steps.

    Returns:
    - tuple: Input sequences (X) and output sequences (y) as numpy arrays.
    """
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        if out_end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return X, y

def uni_split_sequence(sequence, n_steps):
    """
    Split univariate sequence into input and output parts.

    Parameters:
    - sequence (list): Univariate time series data.
    - n_steps (int): Number of input time steps.

    Returns:
    - tuple: Input sequences (X) and output sequences (y) as numpy arrays.
    """
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return X , y


def multivariate_split_sequence(sequence, n_steps):
    """
    Split multivariate sequence into input and output parts.

    Parameters:
    - sequence (numpy.ndarray): Multivariate time series data.
    - n_steps (int): Number of input time steps.

    Returns:
    - tuple: Input sequences (X) and output sequences (y) as numpy arrays.
    """
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x, seq_y = sequence[i:end_ix, :], sequence[end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return X, y



features_columns = ['Lat', 'Lon', 'Speed(knots)']
target_column = 'category'


features = df[features_columns].values
target = df[target_column].values

n_steps = 5

X, y = multivariate_split_sequence(features, n_steps)



def rmse(pred, actual):
    """
    Calculate the Root Mean Squared Error (RMSE) between two arrays.

    Parameters:
    - pred (numpy.ndarray): Predicted values.
    - actual (numpy.ndarray): Actual values.

    Returns:
    - float: Root Mean Squared Error.
    """
    return np.sqrt(((pred - actual) ** 2).mean())
def categorical(pred, actual):
    """
    Compute classification metrics for categorical values.

    Parameters:
    - pred (numpy.ndarray): Predicted categorical values.
    - actual (numpy.ndarray): Actual categorical values.

    Returns:
    - tuple: Accuracy, AUC, Confusion Matrix, Precision, Recall, F1 Score.
    """
    cm = confusion_matrix(pred, actual)
    acc = accuracy_score(actual, pred, normalize=True, sample_weight=None)
    ps1 = precision_score(pred, actual, average='micro')
    rs1 = recall_score(pred, actual, average='micro')
    f11 = f1_score(pred, actual, average='micro')
    auc = roc_auc_score(actual, pred)
    return acc, auc, cm, ps1, rs1, f11
def make_confusion_matrix_chart(cf_matrix_test, cmap='Blues', annot_kws=None):
    """
    Generate and display a heatmap-style confusion matrix chart.

    Parameters:
    - cf_matrix_test (numpy.ndarray): Confusion matrix.
    - cmap (str): Colormap for the heatmap.
    - annot_kws (dict): Additional keyword arguments for annotation customization.

    Returns:
    - None
    """
    # Set up the figure and axes
    plt.figure(figsize=(12, 6))

    # Customize the heatmap using seaborn
    sns.heatmap(cf_matrix_test, annot=True, cmap=cmap,
                yticklabels=['0', '1'], xticklabels=['0', '1'],
                fmt='g', annot_kws=annot_kws)

    # Customize axis labels and title
    plt.ylabel("Actual")
    plt.xlabel("Predicted")
    plt.title('Confusion Matrix - Test Data')

    return None

In [ ]:
# Define whether it's a univariate or multivariate case
univariate_1 = False  # If True, it's a multivariate case

# Define sequence and time step parameters
n_steps_in_1 = 8
n_seq_1= 2
n_steps_out_1 = 1  # Number of output time steps

# Define the number of features for input and output
n_features_in_1 = 2  # Latitude, Longitude
n_features_out_1 = 2  # 'category'

# Define whether it's a univariate or multivariate case
univariate_2 = True

# Define sequence and time step parameters
n_steps_in_2  = 8
n_seq_2 = 2
n_steps_out_2 = 1  # Number of output time steps

# Define the number of features for input and output
n_features_in_2 = 1
n_features_out_2 = 2

hidden_layers = 50


epochs = 100
No_exp = 1




In [ ]:
# Initialize variables
cyclone_id = df['No. of Cycl'][0]
X = []
Y = []
start_index = 0
end_index = 0

# Iterate through the DataFrame
for i in range(1, df.shape[0]):
    # Check if the cyclone ID is the same as the previous row
    if df['No. of Cycl'][i] == cyclone_id:
        end_index += 1
    else:
        # Split the sequence and append to X and Y
        x, y = multivariate_split_sequence(features[start_index:end_index + 1, :], n_steps_in_1)
        X.append(x)
        Y.append(y)

        # Update start and end indices for the new cyclone
        cyclone_id = df['No. of Cycl'][i]
        start_index = i
        end_index = i
# Check if it's the last row of the DataFrame
    if i == df.shape[0] - 1:
        # Split the sequence and append to X and Y
        x, y = multivariate_split_sequence(features[start_index:end_index + 1, :], n_steps_in_1)
        X.append(x)
        Y.append(y)


In [ ]:
# Print the initial lengths of X and Y
print(len(X), len(Y))
# Flattening X and Y
X = [item for sublist in X for item in sublist]
Y = [item for sublist in Y for item in sublist]
# Print lengths of X and Y after flattening
print(len(X), len(Y))
print(type(X), Y[0])
print(speed[:10])


670 670
18863 18863
<class 'list'> 25.0
[20, 25, 25, 25, 25, 30, 30, 30, 25, 25]


In [ ]:
for i in range(len(X)):
  X[i] = X[i].tolist()
for i in range(len(X)):
  print(type(X[i]))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

In [ ]:
# Initialize 'intensify_y' with zeros
intensify_y = [0] * len(X)

# Add labels for rapid intensification
for i in range(len(X)):
    for j in range(len(X[0]) - 4, len(X[0])):
        if (Y[i] - X[i][j][2]) >= 30:
            intensify_y[i] = 1
            break

print(len(intensify_y))

Y = intensify_y


18863


In [ ]:
Xe = X
Ye = Y
print(X[0])


[[11.8, 86.7, 20.0], [11.600000000000001, 86.4, 25.0], [11.3, 86.2, 25.0], [10.9, 86.2, 25.0], [10.4, 86.10000000000001, 25.0], [10.100000000000001, 85.7, 30.0], [10.0, 85.30000000000001, 30.0], [10.3, 85.10000000000001, 30.0]]


In [ ]:
from collections import Counter

import numpy as np


# Flatten the time steps and features
X1 = np.array(X)
X_flat = np.reshape(X1, (X1.shape[0], -1))

# Check the class distribution
print("Class distribution:", Counter(Y))

# Reshape the oversampled data back to its original shape
X1 = np.reshape(X_flat, (X_flat.shape[0], X1.shape[1], X1.shape[2]))

X = X1.tolist()

Class distribution: Counter({0: 18217, 1: 646})


In [ ]:
print(X[0][0])

[11.8, 86.7, 20.0]


In [ ]:
for i in range(len(X)):
  for j in range(len(X[i])):
    X[i][j].append(Y[i])
print(X[0][0])

[11.8, 86.7, 20.0, 0]


In [ ]:
X1 = []
Y1 = []
for i in range(len(X)):
  x = []
  y = []
  for j in range(0,4):
    x.append(X[i][j])
  for k in range(4, 8):
    y.append(X[i][k])
  X1.append(x)
  Y1.append(y)
print(X1[0], Y1[0])

[[11.8, 86.7, 20.0, 0], [11.600000000000001, 86.4, 25.0, 0], [11.3, 86.2, 25.0, 0], [10.9, 86.2, 25.0, 0]] [[10.4, 86.10000000000001, 25.0, 0], [10.100000000000001, 85.7, 30.0, 0], [10.0, 85.30000000000001, 30.0, 0], [10.3, 85.10000000000001, 30.0, 0]]


In [ ]:
print(Y1[0])

[[10.4, 86.10000000000001, 25.0, 0], [10.100000000000001, 85.7, 30.0, 0], [10.0, 85.30000000000001, 30.0, 0], [10.3, 85.10000000000001, 30.0, 0]]


In [ ]:
Y2 = []
for i in range(len(Y1)):
  y = []
  for j in range(len(Y1[i])):
    y1 = []
    for k in range(0,3):
      y1.append(Y1[i][j][k])
    y.append(y1)
  Y2.append(y)
print(Y2[0])

[[10.4, 86.10000000000001, 25.0], [10.100000000000001, 85.7, 30.0], [10.0, 85.30000000000001, 30.0], [10.3, 85.10000000000001, 30.0]]


In [ ]:
print(Y2[123])
Y1 = Y2

[[9.200000000000001, 95.0, 35.0], [9.5, 95.10000000000001, 35.0], [9.9, 95.4, 35.0], [10.200000000000001, 95.7, 35.0]]


In [ ]:
print(X1[0])

[[11.8, 86.7, 20.0, 0], [11.600000000000001, 86.4, 25.0, 0], [11.3, 86.2, 25.0, 0], [10.9, 86.2, 25.0, 0]]


In [ ]:
# Calculate the training limit as 70% of the total length of X for data augmentation
train_limit = int(len(X1) * 0.70)

# Display the calculated training limit
print("Training set size:", train_limit)

Training set size: 13204


In [ ]:
# Extract test data for evaluation
test_X = X1[train_limit + 1:]
test_Y = Y1[train_limit + 1:]
X1 = X1[:train_limit]
Y1 = Y1[:train_limit]
# Display the lengths of the datasets
print("Length of X:", len(X1))
print("Length of Y:", len(Y1))
print("Length of Test X (for evaluation):", len(test_X))
print("Length of Test Y (for evaluation):", len(test_Y))


Length of X: 13204
Length of Y: 13204
Length of Test X (for evaluation): 5658
Length of Test Y (for evaluation): 5658


In [ ]:
print(len(X1))

13204


In [ ]:
test_X1 = []
test_Y1 = []
for i in range(len(test_X)):
  if(test_X[i][0][3]==1):
    test_X1.append(test_X[i])
    test_Y1.append(test_Y[i])
test_X = test_X1
test_Y = test_Y1
print(test_X[0])

[[13.700000000000001, 110.7, 55.0, 1], [14.600000000000001, 109.7, 65.0, 1], [14.600000000000001, 109.7, 65.0, 1], [15.4, 108.7, 70.0, 1]]


In [ ]:
print(len(test_X))

114


In [ ]:
print(X1[0], test_X[0])

[[11.8, 86.7, 20.0, 0], [11.600000000000001, 86.4, 25.0, 0], [11.3, 86.2, 25.0, 0], [10.9, 86.2, 25.0, 0]] [[13.700000000000001, 110.7, 55.0, 1], [14.600000000000001, 109.7, 65.0, 1], [14.600000000000001, 109.7, 65.0, 1], [15.4, 108.7, 70.0, 1]]


In [ ]:
# Convert the list to a NumPy array, shuffle it, and then convert it back to a list
idx = np.random.permutation(len(X1))

# Initialize lists to store shuffled data
x_shuffled = []
y_shuffled = []

# Iterate through the shuffled indices
for i in idx:
    # Append shuffled data to the lists
    x_shuffled.append(X1[i])
    y_shuffled.append(Y1[i])

X1 = x_shuffled
Y1 = y_shuffled


In [ ]:
print(len(X1[0]))

4


In [ ]:
num_timesteps = 4
num_features = 4

# Number of output time steps and features
num_output_timesteps = 4
num_output_features = 3



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def simple_lstm_model(num_timesteps, num_output_timesteps, num_output_features, num_input_features):
    """
    Create and train a simpler LSTM model for predicting latitude and longitude.

    Parameters:
    - num_timesteps: Number of input time steps.
    - num_output_timesteps: Number of output time steps.
    - num_output_features: Number of output features (latitude and longitude).
    - num_input_features: Number of input features.

    Returns:
    - LSTM model.
    """

    model = Sequential()
    model.add(LSTM(units=50, input_shape=(num_timesteps, num_input_features), return_sequences=False))
    model.add(Dense(units=num_output_timesteps * num_output_features))
    model.add(tf.keras.layers.Reshape((num_output_timesteps, num_output_features)))

    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def lstm_regression(model_name, x_train, x_test, y_train, y_test, Num_Exp, n_steps_in, n_steps_out, Epochs, Hidden):
    # Initialize arrays to store accuracy metrics
    train_mse_lat = np.zeros(Num_Exp)
    train_mse_lon = np.zeros(Num_Exp)
    test_mse_lat = np.zeros(Num_Exp)
    test_mse_lon = np.zeros(Num_Exp)
    train_mae_lat = np.zeros(Num_Exp)
    train_mae_lon = np.zeros(Num_Exp)
    test_mae_lat = np.zeros(Num_Exp)
    test_mae_lon = np.zeros(Num_Exp)
    train_r2_lat = np.zeros(Num_Exp)
    train_r2_lon = np.zeros(Num_Exp)
    test_r2_lat = np.zeros(Num_Exp)
    test_r2_lon = np.zeros(Num_Exp)

    model = simple_lstm_model(4, 4, 3, 4)

    # Display the model summary
    model.summary()

    # Initialize arrays to store predictions
    Best_mse_lat = float('inf')  # Initialize the best MSE score
    Best_mse_lon = float('inf')
    Best_mae_lat = float('inf')  # Initialize the best MSE score
    Best_mae_lon = float('inf')
    Best_r2_lat = 0  # Initialize the best MSE score
    Best_r2_lon = 0

    # Loop through experiment runs
    start_time = time.time()
    for run in range(Num_Exp):
        print("Experiment", run + 1, "in progress")

        # Fit the model
        model.fit(x_train, y_train, epochs=Epochs, batch_size=10, verbose=0, shuffle=False)
        y_predicttrain = model.predict(x_train)
        y_predicttest = model.predict(x_test)
        print(y_predicttrain[0])
        print(y_predicttest[0])
        print(len(y_predicttrain))
        print(len(y_predicttest))
        # Flatten y_true and y_predict arrays
        y_train_lat = []
        for i in range(len(y_train)):
          for j in range(len(y_train[i])):
            y_train_lat.append(y_train[i][j][0])
        y_predtrain_lat = []
        for i in range(len(y_predicttrain)):
          for j in range(len(y_predicttrain[i])):
            y_predtrain_lat.append(y_predicttrain[i][j][0])
        y_train_lon = []
        for i in range(len(y_train)):
          for j in range(len(y_train[i])):
            y_train_lon.append(y_train[i][j][1])
        y_predtrain_lon = []
        for i in range(len(y_predicttrain)):
          for j in range(len(y_predicttrain[i])):
            y_predtrain_lon.append(y_predicttrain[i][j][1])
        y_test_lat = []
        print(type(y_test[0]))
        print(len(y_test))
        for i in range(len(y_test)):
          for j in range(len(y_test[i])):
            y_test_lat.append(y_test[i][j][0])
        y_predtest_lat = []
        for i in range(len(y_predicttest)):
          for j in range(len(y_predicttest[i])):
            y_predtest_lat.append(y_predicttest[i][j][0])
        y_test_lon = []
        for i in range(len(y_test)):
          for j in range(len(y_test[i])):
            y_test_lon.append(y_test[i][j][1])
        y_predtest_lon = []
        for i in range(len(y_predicttest)):
          for j in range(len(y_predicttest[i])):
            y_predtest_lon.append(y_predicttest[i][j][1])




        # Calculate MSE for train and test sets
        mse_train_lat = np.sqrt(mean_squared_error(y_train_lat, y_predtrain_lat))
        mse_train_lon = np.sqrt(mean_squared_error(y_train_lon, y_predtrain_lon))
        mse_test_lat = np.sqrt(mean_squared_error(y_test_lat, y_predtest_lat))
        mse_test_lon = np.sqrt(mean_squared_error(y_test_lon, y_predtest_lon))
        mae_train_lat = mean_absolute_error(y_train_lat, y_predtrain_lat)
        mae_train_lon = mean_absolute_error(y_train_lon, y_predtrain_lon)
        mae_test_lat = mean_absolute_error(y_test_lat, y_predtest_lat)
        mae_test_lon = mean_absolute_error(y_test_lon, y_predtest_lon)
        r2_train_lat = r2_score(y_train_lat, y_predtrain_lat)
        r2_train_lon = r2_score(y_train_lon, y_predtrain_lon)
        r2_test_lat = r2_score(y_test_lat, y_predtest_lat)
        r2_test_lon = r2_score(y_test_lon, y_predtest_lon)


        # Store MSE in arrays
        train_mse_lat[run] = mse_train_lat
        train_mse_lon[run] = mse_train_lon
        test_mse_lat[run] = mse_test_lat
        test_mse_lon[run] = mse_test_lon
        train_mae_lat[run] = mae_train_lat
        train_mae_lon[run] = mae_train_lon
        test_mae_lat[run] = mae_test_lat
        test_mae_lon[run] = mae_test_lon
        train_r2_lat[run] = r2_train_lat
        train_r2_lon[run] = r2_train_lon
        test_r2_lat[run] = r2_test_lat
        test_r2_lon[run] = r2_test_lon
        print("train MSE latitude: ", mse_train_lat)
        print("train MSE longitude: ", mse_train_lon)
        print("test MSE latitude: ", mse_test_lat)
        print("test  MSE longitude: ", mse_test_lon)
        print("train MAE latitude: ", mae_train_lat)
        print("train MAE longitude: ", mae_train_lon)
        print("test MAE latitude: ", mae_test_lat)
        print("test  MAE longitude: ", mae_test_lon)
        print("train R2 SCORE latitude: ", r2_train_lat)
        print("train R2 SCORE longitude: ", r2_train_lon)
        print("test R2 SCORE latitude: ", r2_test_lat)
        print("test R2 SCORE longitude: ", r2_test_lon)

    # Update the best MSE score and associated predictions
    if mse_test_lat < Best_mse_lat:
        Best_mse_lat = mse_test_lat

    if mse_test_lon < Best_mse_lon:
        Best_mse_lon = mse_test_lon
    if mae_test_lat < Best_mae_lat:
        Best_mae_lat = mae_test_lat
    if mae_test_lon < Best_mae_lon:
        Best_mae_lon = mae_test_lon
    if r2_test_lat > Best_r2_lat:
        Best_r2_lat = r2_test_lat

    if r2_test_lon > Best_r2_lon:
        Best_r2_lon = r2_test_lon




    # Save the trained model
    model.save("model_" + model_name + '.h5')

    # Calculate standard deviations of train and test MSEs
    train_std1_lat = np.std(train_mse_lat)
    train_std1_lon = np.std(train_mse_lon)
    test_std1_lat = np.std(test_mse_lat)
    test_std1_lon = np.std(test_mse_lon)

    train_std2_lat = np.std(train_mae_lat)
    train_std2_lon = np.std(train_mae_lon)
    test_std2_lat = np.std(test_mae_lat)
    test_std2_lon = np.std(test_mae_lon)

    train_std3_lat = np.std(train_r2_lat)
    train_std3_lon = np.std(train_r2_lon)
    test_std3_lat = np.std(test_r2_lat)
    test_std3_lon = np.std(test_r2_lon)


    # Display experiment summary
    print("Total time for", Num_Exp, "experiments", time.time() - start_time)
    print("MSE for latitude test data: ", test_mse_lat)
    print("MSE for longitude test data: ", test_mse_lon)
    print("MSE for latitude test data: ", test_mae_lat)
    print("MSE for longitude test data: ", test_mae_lon)
    print("R2 SCORE for latitude test data: ", test_r2_lat)
    print("R2 SCORE for longitude test data: ", test_r2_lon)
    print("Mean and std MSE latitude: ", np.mean(test_mse_lat), "Std Dev latitude: ", test_std1_lat)
    print("Mean and std MSE longitude: ", np.mean(test_mse_lon), "Std Dev longitude: ", test_std1_lon)
    print("Mean and std latitude: ", np.mean(test_mae_lat), "Std Dev latitude: ", test_std2_lat)
    print("Mean and std longitude: ", np.mean(test_mae_lon), "Std Dev longitude: ", test_std2_lon)
    print("Mean and std latitude: ", np.mean(test_r2_lat), "Std Dev latitude: ", test_std3_lat)
    print("Mean and std longitude: ", np.mean(test_r2_lon), "Std Dev longitude: ", test_std3_lon)


    # Return relevant information
    return train_mse_lat,train_mse_lon,test_mse_lat,test_mse_lon, train_std1_lat, train_std1_lon, test_std1_lat, test_std1_lon,  y_predicttrain, y_predicttest


In [ ]:
# Initialize dictionaries to store results for each model
for j in range(1):

    n_steps_out = j + 1

    print('---------------------------------------------------------')
    print('Number of steps out:', n_steps_out)

    # Loop over different LSTM models
    x_train, y_train = np.asarray(X1), np.asarray(Y1)
    x_test, y_test = np.asarray(test_X), np.asarray(test_Y)
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 4))
    x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 4))

    # Call the LSTM regression function and retrieve results
    train_mse_lat,train_mse_lon,test_mse_lat,  test_mse_lon, train_std_lat, train_std_lon, test_std_lat, test_std_lon, y_predicttrain, y_predicttest = lstm_regression(
        "gentrack", x_train, x_test, y_train, y_test, 1, 5, 3, epochs, hidden_layers)


---------------------------------------------------------
Number of steps out: 1


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        11,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 12)             │           612 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 4, 3)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,612 (45.36 KB)

 Trainable params: 11,612 (45.36 KB)

 Non-trainable params: 0 (0.00 B)

Experiment 1 in progress
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
[[ 19.169098 118.66673  120.777115]
 [ 19.691442 118.481964 118.19889 ]
 [ 20.20286  118.39531  113.857635]
 [ 20.694372 118.39002  109.25196 ]]
[[ 16.246292 107.52583   77.413475]
 [ 16.751337 107.08869   87.15077 ]
 [ 17.194778 106.705536  97.02991 ]
 [ 17.654623 106.402725 106.12959 ]]
13204
114
<class 'numpy.ndarray'>
114


train MSE latitude:  1.0594000833547035
train MSE longitude:  1.5372001765563061
test MSE latitude:  1.1287557412062639
test  MSE longitude:  2.069510609077346
train MAE latitude:  0.7977634943974299
train MAE longitude:  1.1536582164206823
test MAE latitude:  0.9512876786683729
test  MAE longitude:  1.5368386937860867
train R2 SCORE latitude:  0.9538649521408131
train R2 SCORE longitude:  0.9964942438505517
test R2 SCORE latitude:  0.7877478997114372
test R2 SCORE longitude:  0.9922211433587268
Total time for 1 experiments 544.6287248134613
MSE for latitude test data:  [1.12875574]
MSE for longitude test data:  [2.06951061]
MSE for latitude test data:  [0.95128768]
MSE for longitude test data:  [1.53683869]
R2 SCORE for latitude test data:  [0.7877479]
R2 SCORE for longitude test data:  [0.99222114]
Mean and std MSE latitude:  1.1287557412062639 Std Dev latitude:  0.0
Mean and std MSE longitude:  2.069510609077346 Std Dev longitude:  0.0
Mean and std latitude:  0.9512876786683729 Std 

In [ ]:
print(y_predicttest[0])
print(y_predicttrain[0])
print(len(y_predicttest))


[[ 16.246292 107.52583   77.413475]
 [ 16.751337 107.08869   87.15077 ]
 [ 17.194778 106.705536  97.02991 ]
 [ 17.654623 106.402725 106.12959 ]]
[[ 19.169098 118.66673  120.777115]
 [ 19.691442 118.481964 118.19889 ]
 [ 20.20286  118.39531  113.857635]
 [ 20.694372 118.39002  109.25196 ]]
114


In [ ]:
print(len(Xe))

18863


In [ ]:
for i in range(len(test_X)):
    for j in range(len(test_X[i])):
        # Remove the last element (1) from each sublist
        test_X[i][j] = test_X[i][j][:-1]
print(test_X)


[[[13.700000000000001, 110.7, 55.0], [14.600000000000001, 109.7, 65.0], [14.600000000000001, 109.7, 65.0], [15.4, 108.7, 70.0]], [[10.3, 71.4, 25.0], [11.100000000000001, 71.4, 30.0], [11.9, 71.2, 30.0], [12.600000000000001, 70.9, 40.0]], [[11.100000000000001, 71.4, 30.0], [11.9, 71.2, 30.0], [12.600000000000001, 70.9, 40.0], [12.8, 70.7, 50.0]], [[11.9, 71.2, 30.0], [12.600000000000001, 70.9, 40.0], [12.8, 70.7, 50.0], [12.9, 70.60000000000001, 60.0]], [[8.700000000000001, 81.60000000000001, 35.0], [8.9, 80.5, 40.0], [9.1, 79.4, 45.0], [9.700000000000001, 78.4, 55.0]], [[8.9, 80.5, 40.0], [9.1, 79.4, 45.0], [9.700000000000001, 78.4, 55.0], [9.700000000000001, 78.4, 55.0]], [[9.1, 79.4, 45.0], [9.700000000000001, 78.4, 55.0], [9.700000000000001, 78.4, 55.0], [10.4, 77.4, 65.0]], [[9.700000000000001, 78.4, 55.0], [9.700000000000001, 78.4, 55.0], [10.4, 77.4, 65.0], [10.4, 77.4, 65.0]], [[13.8, 126.30000000000001, 65.0], [13.8, 126.30000000000001, 65.0], [14.3, 125.60000000000001, 75.0],

In [ ]:
for i in range(len(test_X)):
  for j in range(4):
    test_X[i].append(y_predicttest[i][j])
print(len(test_X[0]))


8


In [ ]:
for i in range(8):
  print(test_X[i])
test_Xbfore = test_X

[[13.700000000000001, 110.7, 55.0], [14.600000000000001, 109.7, 65.0], [14.600000000000001, 109.7, 65.0], [15.4, 108.7, 70.0], array([ 16.246292, 107.52583 ,  77.413475], dtype=float32), array([ 16.751337, 107.08869 ,  87.15077 ], dtype=float32), array([ 17.194778, 106.705536,  97.02991 ], dtype=float32), array([ 17.654623, 106.402725, 106.12959 ], dtype=float32)]
[[10.3, 71.4, 25.0], [11.100000000000001, 71.4, 30.0], [11.9, 71.2, 30.0], [12.600000000000001, 70.9, 40.0], array([13.74887 , 69.29438 , 47.007965], dtype=float32), array([14.208134, 69.00216 , 57.381332], dtype=float32), array([14.695488, 68.62461 , 68.03616 ], dtype=float32), array([15.044872, 68.03498 , 77.83523 ], dtype=float32)]
[[11.100000000000001, 71.4, 30.0], [11.9, 71.2, 30.0], [12.600000000000001, 70.9, 40.0], [12.8, 70.7, 50.0], array([13.43579 , 70.76658 , 56.597748], dtype=float32), array([13.853321, 70.44705 , 67.87321 ], dtype=float32), array([14.297781, 70.06747 , 79.1626  ], dtype=float32), array([14.654995

In [ ]:
Ym = []

for i in range(len(Xe)):
  Ym.append(Xe[i][7][2])
  Xe[i] = Xe[i][:-1]

In [ ]:
intensify_y = [0]*len(Xe)
for i in range(len(Xe)):
    for j in range(len(Xe[0]) - 4, len(Xe[0])):
        if (Ym[i] - Xe[i][j][2]) >= 30:
            intensify_y[i] = 1
            break


print(len(intensify_y))


Ye = intensify_y

18863


In [ ]:
# Convert the list to a NumPy array, shuffle it, and then convert it back to a list
idx = np.random.permutation(len(Xe))

# Initialize lists to store shuffled data
x_shuffled = []
y_shuffled = []

# Iterate through the shuffled indices
for i in idx:
    # Append shuffled data to the lists
    x_shuffled.append(Xe[i])
    y_shuffled.append(Ye[i])

Xe = x_shuffled
Ye = y_shuffled

In [ ]:
# Calculate the training limit as 75% of the total length of X
train_limit_gen = int(len(Xe) * 0.75)

# Display the calculated training limit
print("Training set size:", train_limit_gen)

Training set size: 14147


In [ ]:
# Extract test data for evaluation
test_X_org = Xe[train_limit_gen + 1:]
test_Y_org = Ye[train_limit_gen + 1:]

# Display the lengths of the datasets
print("Length of X:", len(Xe))
print("Length of Y:", len(Ye))
print("Length of Test X (for evaluation):", len(test_X_org))
print("Length of Test Y (for evaluation):", len(test_Y_org))

Length of X: 18863
Length of Y: 18863
Length of Test X (for evaluation): 4715
Length of Test Y (for evaluation): 4715


In [ ]:
speed_x = Xe
test_Xe = Xe[train_limit_gen+1:]
test_Xe = np.asarray(test_Xe).astype(float)
test_Ye = Ye[train_limit_gen+1:]
Xe = Xe[:train_limit_gen]
Xe = np.asarray(Xe).astype(float)
Ye = Ye[:train_limit_gen]
print(len(test_Xe), len(test_Ye))
len(Xe), len(Ye)

4715 4715


(14147, 14147)

In [ ]:
Xe1 = np.array(Xe)
Ye1 = np.array(Ye)
print(len(Ye1))
Xe_positive1 = Xe1[Ye1 == 1]
print(Xe_positive1.shape)

14147
(513, 7, 3)


In [ ]:
print(test_X[0])

[[13.700000000000001, 110.7, 55.0], [14.600000000000001, 109.7, 65.0], [14.600000000000001, 109.7, 65.0], [15.4, 108.7, 70.0], array([ 16.246292, 107.52583 ,  77.413475], dtype=float32), array([ 16.751337, 107.08869 ,  87.15077 ], dtype=float32), array([ 17.194778, 106.705536,  97.02991 ], dtype=float32), array([ 17.654623, 106.402725, 106.12959 ], dtype=float32)]


In [ ]:
print(len(test_X))

114


In [ ]:
print(len(Xe[0]))

7


In [ ]:
for i in range(8):
  print(Xe[i])

[[21.4 82.1 30. ]
 [21.4 81.9 30. ]
 [21.8 81.3 30. ]
 [22.  80.4 35. ]
 [21.8 79.4 35. ]
 [21.4 78.6 30. ]
 [20.9 78.  30. ]]
[[ 17.9 117.9 115. ]
 [ 17.9 117.9 115. ]
 [ 18.3 117.3 115. ]
 [ 18.3 117.3 115. ]
 [ 18.3 117.3 115. ]
 [ 18.9 116.9 105. ]
 [ 18.9 116.9 105. ]]
[[ 9.6 94.1 30. ]
 [ 9.9 93.7 25. ]
 [10.2 93.3 25. ]
 [10.3 92.9 25. ]
 [10.4 92.7 25. ]
 [10.4 92.5 25. ]
 [10.4 92.3 25. ]]
[[ 13.8 126.   15. ]
 [ 13.8 125.3  15. ]
 [ 13.9 124.7  20. ]
 [ 13.9 124.   20. ]
 [ 14.  122.8  25. ]
 [ 14.1 121.1  25. ]
 [ 14.5 119.3  25. ]]
[[12.2 59.9 55. ]
 [12.2 59.9 55. ]
 [12.6 59.5 65. ]
 [12.6 59.5 65. ]
 [13.2 59.2 90. ]
 [13.2 59.2 90. ]
 [13.2 59.2 90. ]]
[[ 6.4 92.6 15. ]
 [ 6.3 92.  15. ]
 [ 6.6 91.4 15. ]
 [ 7.1 91.1 25. ]
 [ 7.8 91.2 25. ]
 [ 8.4 91.6 30. ]
 [ 9.3 91.4 40. ]]
[[20.8 57.7 45. ]
 [21.8 56.8 45. ]
 [22.8 55.5 45. ]
 [23.4 54.7 45. ]
 [24.  53.7 45. ]
 [24.5 53.2 35. ]
 [25.3 53.1 30. ]]
[[ 12.7  84.7 115. ]
 [ 12.8  84.1 110. ]
 [ 12.8  83.6 105. ]
 [ 12.

In [ ]:
print(Xe[0])

[[21.4 82.1 30. ]
 [21.4 81.9 30. ]
 [21.8 81.3 30. ]
 [22.  80.4 35. ]
 [21.8 79.4 35. ]
 [21.4 78.6 30. ]
 [20.9 78.  30. ]]


In [ ]:
X_bfore = Xe

In [ ]:
# Ensure test_X is a list of lists
test_X = list(test_X)

# Remove the 8th element (index 7) from each list in test_X
for i in range(len(test_X)):
    test_X[i] = test_X[i][:7] + test_X[i][8:]  # Exclude the 8th element

# Print to verify
print(test_X[0])  # Check if the 8th element is removed


[[13.700000000000001, 110.7, 55.0], [14.600000000000001, 109.7, 65.0], [14.600000000000001, 109.7, 65.0], [15.4, 108.7, 70.0], array([ 16.246292, 107.52583 ,  77.413475], dtype=float32), array([ 16.751337, 107.08869 ,  87.15077 ], dtype=float32), array([ 17.194778, 106.705536,  97.02991 ], dtype=float32)]


In [ ]:
print(len(X_bfore))

14147


In [ ]:
print(len(Xe))
print(len(Ye))

14147
14147


In [ ]:
Xe = list(Xe)  # Convert Xe to a list before the loop

for i in range(len(test_X)):
    Xe.append(test_X[i])
    Ye.append(1) # Append elements

Xe = np.array(Xe)  # Convert back to NumPy array if needed


In [ ]:
print(len(Xe))

14261


In [ ]:
print(len(test_X))

114


In [ ]:
print(len(Xe))

14261


In [ ]:

univariate = False
# Define sequence and time step parameters
n_steps_in = 7
n_seq = 3
n_steps_out = 1  # Number of output time steps

# Define the number of features for input and output
n_features_in = 3  # Latitude, Longitude, Speed
n_features_out = 2  # 'category'

# Define the number of hidden layers in the model
hidden_layers = 50

# Define training parameters
epochs = 100
No_exp = 30 # Number of experiments

# Display the configuration
print(f"Univariate: {univariate}")
print(f"Number of Input Time Steps: {n_steps_in}")
print(f"Number of Input Sequences: {n_seq}")
print(f"Number of Output Time Steps: {n_steps_out}")
print(f"Number of Input Features: {n_features_in}")
print(f"Number of Output Features: {n_features_out}")
print(f"Number of Hidden Layers: {hidden_layers}")
print(f"Epochs: {epochs}")
print(f"Number of Experiments: {No_exp}")



Univariate: False
Number of Input Time Steps: 7
Number of Input Sequences: 3
Number of Output Time Steps: 1
Number of Input Features: 3
Number of Output Features: 2
Number of Hidden Layers: 50
Epochs: 100
Number of Experiments: 30


In [ ]:
def multivariate_vanilla_lstm(n_steps_in, n_steps_out, n_features_in, n_features_out):
    """
    Create a Multivariate LSTM model for sequence prediction.

    Parameters:
    - n_steps_in: Number of time steps in the input sequence.
    - n_steps_out: Number of time steps in the output sequence.
    - n_features_in: Number of input features.
    - n_features_out: Number of output features.

    Returns:
    - model: Compiled Multivariate LSTM model.
    """
    # Initialize a sequential model
    model = Sequential()

    # Add an LSTM layer with 50 units and ReLU activation
    model.add(LSTM(50, activation='relu', input_shape=(n_steps_in, n_features_in)))

    # Add a Dense layer with a single unit and sigmoid activation for binary classification
    model.add(Dense(2, activation='softmax'))

    # Compile the model using Adam optimizer and binary crossentropy loss
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, precision_recall_curve, auc

def lstm(model_name, method, univariate, x_train, x_test, y_train, y_test, Num_Exp, n_steps_in, n_steps_out, Epochs, Hidden):
    train_acc = np.zeros(Num_Exp)
    test_acc = np.zeros(Num_Exp)
    train_losses = []
    test_losses = []

    # Choose the model
    if model_name == 'vanilla':
        model = multivariate_vanilla_lstm(n_steps_in, n_steps_out, n_features_in, n_features_out)


    model.summary()

    y_predicttest_allruns = np.zeros([Num_Exp, x_test.shape[0], y_test.shape[1]])
    Best_f1 = 0
    act_test = [y_test[i].argmax() for i in range(y_test.shape[0])]
    act_train = [y_train[i].argmax() for i in range(y_train.shape[0])]

    Best_report_train = dict()
    Best_report_test = dict()
    all_report_train = dict()
    all_report_test = dict()

    start_time = time.time()
    for run in range(Num_Exp):
        print("Experiment", run + 1, "in progress")
        history = model.fit(x_train, y_train, epochs=Epochs, batch_size=10, verbose=0, shuffle=False, validation_data=(x_test, y_test))

        train_losses.append(history.history['loss'])
        test_losses.append(history.history['val_loss'])

        y_predicttrain = model.predict(x_train)
        y_predicttest = model.predict(x_test)

        pred_test = [y_predicttest[i].argmax() for i in range(y_predicttest.shape[0])]
        pred_train = [y_predicttrain[i].argmax() for i in range(y_predicttrain.shape[0])]

        report_train = classification_report(act_train, pred_train, labels=[0, 1], output_dict=True)
        report_test = classification_report(act_test, pred_test, labels=[0, 1], output_dict=True)

        all_report_train[run] = report_train
        all_report_test[run] = report_test

        test_acc[run] = report_test['1']['f1-score']
        print("train acc: ", report_train['1']['f1-score'])
        print("test acc: ", test_acc[run])

        if test_acc[run] > Best_f1:
            Best_f1 = test_acc[run]
            Best_Predict_Test = y_predicttest
            Best_report_train, Best_report_test = report_train, report_test

    model.save("model_" + model_name + "_" + method + '.h5')

    train_std = np.std(train_acc)
    test_std = np.std(test_acc)

    print("Total time for", Num_Exp, "experiments", time.time() - start_time)
    print("F1 scores for test data: ", test_acc)
    print("Mean: ", np.mean(test_acc), "Std Dev: ", test_std)

    # Plot loss curves
    plt.figure(figsize=(12, 6))
    plt.plot(np.mean(train_losses, axis=0), label='Train Loss')
    plt.plot(np.mean(test_losses, axis=0), label='Test Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Testing Loss Curves')
    plt.legend()
    plt.show()

    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(act_test, Best_Predict_Test.argmax(axis=1))
    pr_auc = auc(recall, precision)

    plt.figure(figsize=(6, 6))
    plt.plot(recall, precision, marker='.', label=f'PR Curve (AUC={pr_auc:.3f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend()
    plt.show()

    return train_acc, test_acc, train_std, test_std, Best_Predict_Test, y_predicttrain, y_predicttest, all_report_train, all_report_test


In [ ]:
print(len(Xe))
print(len(Ye))

14261
14261


In [ ]:
# Create a random permutation of indices for shuffling
idx = np.random.permutation(len(Xe))

# Initialize lists to store shuffled data
x_shuffled = []
y_shuffled = []

# Iterate through the shuffled indices
for i in idx:
    # Append shuffled data to the lists
    x_shuffled.append(Xe[i])
    y_shuffled.append(Ye[i])
x_a = np.array(x_shuffled)
print(x_a.shape)

(14261, 7, 3)


In [ ]:
# Convert the shuffled labels to one-hot encoded format for training data
Y_hot_encoded_train = np.asarray(to_categorical(y_shuffled))

# Convert the test labels to one-hot encoded format
Y_hot_encoded_test = np.asarray(to_categorical(test_Ye))

# Print the shapes of the one-hot encoded training and test labels
print(Y_hot_encoded_train.shape, Y_hot_encoded_test.shape)

(14261, 2) (4715, 2)


In [ ]:
x_shuffled[0], test_Xe[0]

(array([[ 16.8, 117.8,  90. ],
        [ 17.7, 118.9,  90. ],
        [ 17.7, 118.9,  90. ],
        [ 17.7, 118.9,  90. ],
        [ 18.6, 119.7,  95. ],
        [ 18.6, 119.7,  95. ],
        [ 18.6, 119.7,  95. ]]),
 array([[ 14.2,  81.7, 105. ],
        [ 14.2,  81.7, 105. ],
        [ 14.2,  81.7, 105. ],
        [ 14.4,  82.1, 120. ],
        [ 14.4,  82.1, 120. ],
        [ 14.4,  82.1, 120. ],
        [ 14.5,  82.3, 120. ]]))

In [ ]:
# Testing on all 4 different kinds of LSTMs
models = ['vanilla']


# Initialize dictionaries to store predictions, actual values, and metrics for training and testing
predictions_train = dict()
actual_train = dict()
predictions_test = dict()
actual_test = dict()
metrics_train = dict()
metrics_test = dict()
test_acc_all = dict()
test_stddev = dict()


In [ ]:
# Initialize dictionaries to store results for each model
predictions_train = dict()
actual_train = dict()
predictions_test = dict()
actual_test = dict()
metrics_train = dict()
metrics_test = dict()
test_acc_all = dict()
test_stddev = dict()

for j in range(1):

    predictions_train_per_step = dict()
    actual_train_per_step = dict()
    predictions_test_per_step = dict()
    actual_test_per_step = dict()
    metrics_train_per_step = dict()
    metrics_test_per_step = dict()
    test_acc_per_step = dict()
    test_stddev_per_step = dict()
    n_steps_out = j + 1

    print('---------------------------------------------------------')
    print('Number of steps out:', n_steps_out)

    # Loop over different LSTM models
    for i in models:
        print("For " + i + ":")

        # Reshape data based on the LSTM model type
        if i in ['vanilla']:
            x_train, y_train = np.asarray(x_shuffled), np.asarray(Y_hot_encoded_train)
            x_test, y_test = np.asarray(test_Xe), np.asarray(Y_hot_encoded_test)
            x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], n_features_in))
            x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], n_features_in))


        # Call the LSTM function and retrieve results
        train_acc, test_acc, train_std_dev, test_std_dev, Best_Predict_Test, y_predicttrain, y_predicttest, report_train, report_test = lstm(
            i, 'original', univariate, x_train, x_test, y_train, y_test, No_exp, n_steps_in, n_steps_out, epochs, hidden_layers)

        # Store results in respective dictionaries
        predictions_train_per_step[i] = y_predicttrain
        actual_train_per_step[i] = y_train
        predictions_test_per_step[i] = Best_Predict_Test
        actual_test_per_step[i] = y_test
        metrics_train_per_step[i] = report_train
        metrics_test_per_step[i] = report_test
        test_acc_per_step[i] = test_acc
        test_stddev_per_step[i] = test_std_dev

    # Store results for the current n_steps_out in the overall dictionaries
    predictions_train[str(j + 1)] = predictions_train_per_step
    actual_train[str(j + 1)] = actual_train_per_step
    predictions_test[str(j + 1)] = predictions_test_per_step
    actual_test[str(j + 1)] = actual_test_per_step
    metrics_train[str(j + 1)] = metrics_train_per_step
    metrics_test[str(j + 1)] = metrics_test_per_step
    test_acc_all[str(j + 1)] = test_acc_per_step
    test_stddev[str(j + 1)] = test_stddev_per_step


---------------------------------------------------------
Number of steps out: 1
For vanilla:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 50)             │        10,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           102 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,902 (42.59 KB)

 Trainable params: 10,902 (42.59 KB)

 Non-trainable params: 0 (0.00 B)

Experiment 1 in progress
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
train acc:  0.6833976833976834
test acc:  0.625
Experiment 2 in progress
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
train acc:  0.6988847583643123
test acc:  0.6592592592592592
Experiment 3 in progress
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
train acc:  0.6580259222333001
test acc:  0.6587301587301587
Experiment 4 in progress


In [ ]:
def make_confusion_matrix_chart(cf_matrix_test, cmap='Blues', annot_kws=None):
    """
    Generate and display a heatmap-style confusion matrix chart.

    Parameters:
    - cf_matrix_test (numpy.ndarray): Confusion matrix.
    - cmap (str): Colormap for the heatmap.
    - annot_kws (dict): Additional keyword arguments for annotation customization.

    Returns:
    - None
    """
    # Set up the figure and axes
    plt.figure(figsize=(12, 6))

    # Customize the heatmap using seaborn
    sns.heatmap(cf_matrix_test, annot=True, cmap=cmap,
                yticklabels=['0', '1'], xticklabels=['0', '1'],
                fmt='g', annot_kws=annot_kws)

    # Customize axis labels and title
    plt.ylabel("Actual")
    plt.xlabel("Predicted")
    plt.title('Confusion Matrix - Test Data')

    return None

In [ ]:
y = [i.argmax() for i in actual_test_per_step['vanilla']]
pred = [i.argmax() for i in predictions_test_per_step['vanilla']]
cf_matrix_test = confusion_matrix(y, pred)
make_confusion_matrix_chart(cf_matrix_test)


In [ ]:
# Initialize lists to store precision, recall, and F1-score metrics for class 0, class 1, overall accuracy,
# macro average, and weighted average across multiple experiments
precision0 = []
precision1 = []
precisionacc = []
precisionmacavg = []
precisionweighavg = []
recall0 = []
recall1 = []
recallacc = []
recallmacavg = []
recallweighavg = []
f10 = []
f11 = []
f1acc = []
f1macavg = []
f1weighavg = []

# Loop through the results of multiple experiments
for i in range(No_exp):
    # Append precision, recall, and F1-score metrics for class 0, class 1, overall accuracy,
    # macro average, and weighted average from the model results
    precision0.append(metrics_test_per_step['vanilla'][i]['0']['precision'])
    precision1.append(metrics_test_per_step['vanilla'][i]['1']['precision'])
    precisionacc.append(metrics_test_per_step['vanilla'][i]['accuracy'])
    precisionmacavg.append(metrics_test_per_step['vanilla'][i]['macro avg']['precision'])
    precisionweighavg.append(metrics_test_per_step['vanilla'][i]['weighted avg']['precision'])

    recall0.append(metrics_test_per_step['vanilla'][i]['0']['recall'])
    recall1.append(metrics_test_per_step['vanilla'][i]['1']['recall'])
    recallacc.append(metrics_test_per_step['vanilla'][i]['accuracy'])
    recallmacavg.append(metrics_test_per_step['vanilla'][i]['macro avg']['recall'])
    recallweighavg.append(metrics_test_per_step['vanilla'][i]['weighted avg']['recall'])

    f10.append(metrics_test_per_step['vanilla'][i]['0']['f1-score'])
    f11.append(metrics_test_per_step['vanilla'][i]['1']['f1-score'])
    f1acc.append(metrics_test_per_step['vanilla'][i]['accuracy'])
    f1macavg.append(metrics_test_per_step['vanilla'][i]['macro avg']['f1-score'])
    f1weighavg.append(metrics_test_per_step['vanilla'][i]['weighted avg']['f1-score'])


In [ ]:
print(str(round(np.mean(precision0),4)) + "±" + str(round(np.std(precision0),4))," & " + str(round(np.mean(recall0),4)) + "±" + str(round(np.std(recall0),4)), " & " + str(round(np.mean(f10),4)) + "±" + str(round(np.std(f10),4)))
print(str(round(np.mean(precision1),4)) + "±" + str(round(np.std(precision1),4))," & " + str(round(np.mean(recall1),4)) + "±" + str(round(np.std(recall1),4)), " & " + str(round(np.mean(f11),4)) + "±" + str(round(np.std(f11),4)))
print(str(round(np.mean(precisionacc),4)) + "±" + str(round(np.std(precisionacc),4))," & " + str(round(np.mean(recallacc),4)) + "±" + str(round(np.std(recallacc),4)), " & " + str(round(np.mean(f1acc),4)) + "±" + str(round(np.std(f1acc),4)))
print(str(round(np.mean(precisionmacavg),4)) + "±" + str(round(np.std(precisionmacavg),4))," & " + str(round(np.mean(recallmacavg),4)) + "±" + str(round(np.std(recallmacavg),4)), " & " + str(round(np.mean(f1macavg),4)) + "±" + str(round(np.std(f1macavg),4)))
print(str(round(np.mean(precisionweighavg),4)) + "±" + str(round(np.std(precisionweighavg),4))," & " + str(round(np.mean(recallweighavg),4)) + "±" + str(round(np.std(recallweighavg),4)), " & " + str(round(np.mean(f1weighavg),4)) + "±" + str(round(np.std(f1weighavg),4)))
